# GRASS Scripting with Python

This is an introduction to using *GRASS* in the *Python* scripting language. This document is a *Jupyter Notebook*, but unlike the [jupyter_example.ipynb](jupyter_example.ipynb) notebook, the examples in this notebook are meant to be used primarily outside of a notebook.

Examples here are using a sample GRASS dataset for North Carolina, USA. The dataset is included in this environment. The interactive notebook is available online thanks to the [*Binder*](https://mybinder.org/) service.

## Usage

To run the selected part which is called a cell, hit `Shift + Enter`.

## Start

There are several ways to use GRASS. When using Python in a notebook, we usually find GRASS Python packages first, import them, initialize GRASS session, and set several variables useful for using GRASS in a notebook.

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import sys
import subprocess
from IPython.display import Image

# Ask GRASS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS packages we need.
import grass.script as gs

# Create a GRASS session.
session = gs.setup.init("~/data/nc_basic_spm_grass7/user1")

# We want functions to raise exceptions and see standard output of the modules in the notebook.
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)
# Simply overwrite existing maps like we overwrite Python variable values.
os.environ["GRASS_OVERWRITE"] = "1"
# Enable map rendering in a notebook.
os.environ["GRASS_FONT"] = "sans"
# Set display modules to render into a file (named map.png by default)
os.environ["GRASS_RENDER_IMMEDIATE"] = "cairo"
os.environ["GRASS_RENDER_FILE_READ"] = "TRUE"
os.environ["GRASS_LEGEND_FILE"] = "legend.txt"

## Raster buffer

Set computational region and create multiple buffers in given distances
around lakes represented as raster:

In [ ]:
gs.parse_command("g.region", raster="lakes", flags="pg")
gs.run_command(
    "r.buffer", input="lakes", output="lakes_buff", distances=[60, 120, 240, 500]
)
gs.run_command("d.erase")
gs.run_command("d.rast", map="lakes_buff")
gs.run_command("d.legend", raster="lakes_buff", range=(2, 5), at=(80, 100, 2, 10))
Image(filename="map.png")

## Vector buffer

Create a negative buffer around state boundary represented as a vector.
Vector modules typically don't follow computational region,
but we set it to inform display modules about our area of interest.

In [ ]:
gs.run_command("v.buffer", input="boundary_state", output="buffer", distance=-10000)
gs.parse_command("g.region", vector="boundary_state", flags="pg")
gs.run_command("d.erase")  # erase the display before drawing again
!rm -f $GRASS_LEGEND_FILE  # and remove the legend file
gs.run_command(
    "d.vect", map="boundary_state", fill_color="#5A91ED", legend_label="State boundary"
)
gs.run_command(
    "d.vect", map="buffer", fill_color="#F8766D", legend_label="Inner portion"
)
gs.run_command("d.legend.vect", at=(10, 35))
Image(filename="map.png")

## Additional GRASS Information and Tutorials

To find more information on what one can do with GRASS APIs, check out:
    
 - [GRASS Manual](https://grass.osgeo.org/grass-stable/manuals)
    
 - [GRASS Python API Manual](https://grass.osgeo.org/grass-stable/manuals/libpython)

For more Jupyter Notebook GRASS tutorials, visit:
 - [grass-tutorials.osgeo.org](https://grass-tutorials.osgeo.org/)

## What else is in the sample North Carolina dataset?

In [ ]:
print(gs.read_command("g.list", type="all"))

## What other GRASS modules can I try in this notebooks?

In [ ]:
print(gs.read_command("g.search.modules", flags="g"))

## When the work finished

When we are finished working the mapset, we should end the GRASS session using `finish()` which will remove the temporary files created in the background. After the call, GRASS modules can no longer be executed, so the call is commented out in this notebook to allow running all cells and, at the same time, going back and experimenting with the code.

In [ ]:
# Uncomment and run when done.
# session.finish()